<H1>Eigenvalue Frequency Analysis</H1>
Model as per 04a Simple Cantilever beam

In [ ]:
# Input Params
length = 5.0 #m		     Length of beam
point_load = 10.0  #kN   Concentrated load on cantilever tip
model_name = "Cantilever Line Beam" 

In [ ]:
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.21.1")

In [ ]:
# Create a new model
db = lusas.newDatabase("Structural", f"{model_name}.mdl")
# Set the vertical axis
db.setLogicalUpAxis("Z")
# Set the unit system
db.setModelUnits("kN,m,t,s,C")

In [ ]:
''' Create a line'''
# Get the geometry data object
geomData = lusas.geometryData().setAllDefaults()
# Set the coordinates of the first point
geomData.addCoords(0, 0, 0)
# Set the coordinates of the second point
geomData.addCoords(length, 0, 0)
# Create the line object from the geomety data
objs = db.createLine(geomData) # Modeller returns an object set containing all the lines created, in this case we only have 1.
# Get the lines
lines = objs.getObjects("Lines")
# Get the single line from the returned array
beamLine = lines[0]

In [ ]:
''' Create a mesh attribute'''
meshAttr = db.createMeshLine("Beam Mesh")
# Set the element type and size of elements BMI21 = 2Node thick beam elements, 1 = 1m length
meshAttr.setSize("BMI21", 1)

In [ ]:
''' Create a geometric attribute'''
geomAttr = db.createGeometricLine("Beam Geometry")
# Set the beam section properties for the section library
geomAttr.setFromLibrary("UK Sections", "Universal Beams (BS4)", "406x178x74kg UB", 0, 0, 0)

In [ ]:
''' Create a material attribute'''
materialAttr = db.createIsotropicMaterial("Steel", 209_000_000, 0.3, 7.8)
# Set the mass damping constant
materialAttr.setValue("ar", "2.0", 0)

In [ ]:
''' Create a support attribute (fixed) '''
fixedSupport = db.createSupportStructural("Fixed")
# set the freedoms F=Free, R=Restrained
fixedSupport.setStructural("R", "R", "R", "R", "R", "R", "F", "F", "F")

In [ ]:
''' Create a point load attribute '''
loadAttr = db.createLoadingConcentrated("Point load")
loadAttr.setConcentrated(0.0, 0.0, -point_load)

In [ ]:
''' Assign the attributes to the geometry '''
# get the assignment object
assignment = lusas.assignment().setAllDefaults()
# Assign the mesh
meshAttr.assignTo(beamLine, assignment)
# Assign the geometry
geomAttr.assignTo(beamLine, assignment)
# Assign the material
materialAttr.assignTo(beamLine, assignment)
# Assign the loading
loadAttr.assignTo(beamLine.getEndPoint(), assignment)

''' Assign the supports to the points of the line '''
# Assign the fixed support to the first point
fixedSupport.assignTo(beamLine.getStartPoint(), assignment)


Create an eigenvalue frequency analysis

In [ ]:
# Create a new analysis without an initial loadcase
analysis = db.createAnalysisStructural("Eigenvalue Frequency Analysis", False)

# Create a loadcase in the analysis
first_loadcase = db.createLoadcase("Eig Freq", analysis.getName())
# Set the loadcase controls to be a frequency analysis searching for the minimum 5 natural frequencies
first_loadcase.setEigenvalueMaxMinControl("Frequency", "Minimum", 5)
# Set the eiegenvector normalisation procedure to mass for a frequency analysis - This just scales the resulting eigenvector and is the default setting
first_loadcase.getEigenvalueControl().setValue("NormalisationProcedure", "GlobalMass")


Create a second eigenvalue frequency analysis with the point load considered as mass

In [ ]:
# Create a new analysis without an initial loadcase
analysis = db.createAnalysisStructural("Eigenvalue Frequency Analysis With Load", False)
# Create a loadcase in the analysis
first_loadcase = db.createLoadcase("Eig Freq with load", analysis.getName())
# Set the loadcase controls to be a frequency analysis searching for the minimum 5 natural frequencies
first_loadcase.setEigenvalueMaxMinControl("Frequency", "Minimum", 5)
# Set the eiegenvector normalisation procedure to mass for a frequency analysis - This just scales the resulting eigenvector and is the default setting
first_loadcase.getEigenvalueControl().setValue("NormalisationProcedure", "GlobalMass")
# Consider the assigned loading as mass in the eigenvalue analysis
first_loadcase.getEigenvalueControl().setValue("loadToMass", True)
# Create the assignment data to target the new loadcase
assignment = lusas.assignment().setAllDefaults().setLoadset(first_loadcase)
# Assign the loading
loadAttr.assignTo(beamLine.getEndPoint(), assignment)

In [ ]:
# Create a new analysis without an initial loadcase
analysis = db.createAnalysisStructural("Implicit Dynamic Analysis", False)
# Create a loadcase in the analysis
first_loadcase = db.createLoadcase("Dynamic point load", analysis.getName())
# Create the assignment data to target the new loadcase
assignment = lusas.assignment().setAllDefaults().setLoadset(first_loadcase)
# Assign the loading
loadAttr.assignTo(beamLine.getEndPoint(), assignment)

# Add a transient control to the loadcase
first_loadcase.setTransientControl(0)
cntrl = first_loadcase.getTransientControl()
# Set the transient control for an implicit analysis in the time domain with an initial time step of 0.01 seconds
cntrl.setTimeDomainDynamics(0.01, False)
# The total response is 3 seconds
cntrl.setValue("TotalResponseTime", 3.0)


# Solve and open results

In [ ]:
db.getAnalysis("Analysis 1").solve(False)
db.getAnalysis("Eigenvalue Frequency Analysis").solve(False)
db.getAnalysis("Eigenvalue Frequency Analysis With Load").solve(False)
db.openAllResults(False)

# Print Loadcase Results

In [ ]:
loadcase = win32.CastTo(db.getLoadset("Loadcase", 2), "IFLoadcase")
result_loadcases = loadcase.getResultsLoadcases()
# Total mass is the same for all so take the first
print(f'Total Mass = {result_loadcases[0].getValue("TMASS"):.3f}')

# Individual results
for result_loadcase in loadcase.getResultsLoadcases():
    print(result_loadcase.getName())
    print(f'   Eigenvalue = {result_loadcase.getValue("EIGVL"):.0f}')
    print(f'   Frequency  = {result_loadcase.getValue("NATFRQ"):.3f}')
    print(f'   Mass Participation in X,Y,Z = {result_loadcase.getValue("PMASSX"):.3f}, {result_loadcase.getValue("PMASSY"):.3f}, {result_loadcase.getValue("PMASSZ"):.3f}')
